In [ ]:
# Install the correct version of PyTorch for Colab
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install PyTorch Geometric dependencies
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.4.1+cu121.html

# Install PyTorch Geometric
!pip install torch-geometric

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.distributions import MultivariateNormal
import torch.optim as optim

# Data Handling Functions
def read_tsp_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    start_idx = lines.index('NODE_COORD_SECTION\n') + 1
    end_idx = lines.index('EOF\n')
    coordinates = []
    for line in lines[start_idx:end_idx]:
        _, x, y = line.strip().split()
        coordinates.append((float(x), float(y)))

    coordinates = np.array(coordinates)
    return coordinates

def calculate_distance_matrix(coordinates):
    """
    Tính ma trận khoảng cách Euclide 2D giữa các điểm trong `coordinates`.

    Args:
        coordinates (np.ndarray): Mảng numpy có kích thước (n, 2), trong đó n là số điểm,
                                   mỗi điểm có 2 tọa độ (x, y).
    Returns:
        torch.Tensor: Ma trận khoảng cách (n x n) dạng tensor.
    """
    n = coordinates.shape[0]
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dx = coordinates[i, 0] - coordinates[j, 0]
            dy = coordinates[i, 1] - coordinates[j, 1]
            distance_matrix[i, j] = np.sqrt(dx**2 + dy**2)
    return torch.tensor(distance_matrix, dtype=torch.float32)

class TSPDataset(Dataset):
    def __init__(self, dataset_dir, num_cities):
        self.dataset_dir = dataset_dir
        self.num_cities = num_cities
        self.files = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.tsp')]
        self.valid_files = []
        self._filter_files()

    def _filter_files(self):
        for file in self.files:
            coordinates = read_tsp_file(file)
            if coordinates.shape[0] == self.num_cities:
                self.valid_files.append(file)

    def __len__(self):
        return len(self.valid_files)

    def __getitem__(self, idx):
        file = self.valid_files[idx]
        coordinates = read_tsp_file(file)
        distance_matrix = calculate_distance_matrix(coordinates)
        return {
            'coordinates': torch.tensor(coordinates, dtype=torch.float32),
            'distance_matrix': distance_matrix
       }

def create_dataloader(dataset_dir, num_cities, batch_size, shuffle=True, num_workers=4):
    dataset = TSPDataset(dataset_dir, num_cities)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
    return dataloader

# Model Definition
class CityGNN(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32, output_dim=3, num_cities=20):
        super(CityGNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.city_gnn = GCNConv(input_dim, hidden_dim)
        self.edge_gnn = GCNConv(1, hidden_dim)
        self.fc_combine = nn.Linear(3 * hidden_dim, hidden_dim)
        self.fc_output_mean = nn.Linear(hidden_dim, output_dim)
        self.fc_output_cov = nn.Linear(hidden_dim, output_dim * output_dim)
        
        # Precompute edge_index for a fully connected graph
        edge_indices = torch.cartesian_prod(torch.arange(num_cities), torch.arange(num_cities)).t().contiguous()
        self.register_buffer('edge_index', edge_indices)
        
        # Initialize solution_param here to ensure it's on the correct device
        self.solution_param = nn.Parameter(torch.randn(num_cities, hidden_dim))
        
        # Khởi tạo trọng số với hàm init_weights đã sửa
        self.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, GCNConv):
            nn.init.xavier_uniform_(m.lin.weight) 
            if m.lin.bias is not None:
                nn.init.zeros_(m.lin.bias)

    def forward(self, city_positions, distance_matrix, solutions):
        B, n, _ = city_positions.size()
        device = city_positions.device

        # Use precomputed edge_index
        edge_index = self.edge_index.to(device)

        # GCN on city positions
        city_features = F.relu(self.city_gnn(city_positions, edge_index))  # B x n x hidden_dim

        # GCN on edge distances
        dist_flat = distance_matrix.view(B, -1, 1)  # B x (n*n) x 1
        edge_features = F.relu(self.edge_gnn(dist_flat, edge_index))  # B x (n*n) x hidden_dim
        edge_features = edge_features.view(B, n, n, self.hidden_dim).mean(dim=2)  # B x n x hidden_dim

        # Solution features
        solution_features = torch.matmul(solutions, self.solution_param)  # B x K x hidden_dim (K=n)

        # Combine features
        combined_features = torch.cat([city_features, edge_features, solution_features], dim=-1)  # B x n x (3*hidden_dim)
        combined_features = F.relu(self.fc_combine(combined_features))  # B x n x hidden_dim

        # Output mean and covariance
        means = self.fc_output_mean(combined_features)  # B x n x 3
        covariances = self.fc_output_cov(combined_features).view(B, n, 3, 3)  # B x n x 3 x 3
        
        # Sử dụng softplus để đảm bảo giá trị dương
        covariances = F.softplus(covariances)
        # Giới hạn covariance matrices để tránh quá lớn
        covariances = covariances @ covariances.transpose(-1, -2)  # Ensure positive semi-definite
        covariances += torch.eye(3, device=device).unsqueeze(0).unsqueeze(0) * 1e-2  # Add noise for stability
        
        return means, covariances


# Action Sampler
class ActionSampler:
    def __init__(self, gnn_model):
        self.gnn_model = gnn_model

    def sample_action(self, city_positions, distance_matrix, solutions):
        """
        Args:
            city_positions (Tensor): B x n x 2.
            distance_matrix (Tensor): B x n x n.
            solutions (Tensor): B x n x n.

        Returns:
            actions (Tensor): B x n x 3.
            log_probs (Tensor): B x n.
        """
        means, covariances = self.gnn_model(city_positions, distance_matrix, solutions)
        dist = MultivariateNormal(means, covariances)
        sampled_actions = dist.rsample()
        sampled_actions = torch.sigmoid(sampled_actions)
        log_probs = dist.log_prob(sampled_actions)

        return sampled_actions, log_probs

# Fitness Calculation
from scipy.optimize import linear_sum_assignment

import numpy as np
import torch

def compute_fitness_loss(positions, distance_matrix):
    B, K, n = positions.size()
    
    decoded_tours = torch.argsort(positions, dim=2, descending=True)  # B x K x n
    decoded_tours_extended = torch.cat([decoded_tours, decoded_tours[:, :, :1]], dim=2)  # B x K x (n+1)

    city_a = decoded_tours_extended[:, :, :-1]  # B x K x n
    city_b = decoded_tours_extended[:, :, 1:]   # B x K x n

    linear_indices = city_a * n + city_b  # B x K x n

    distance_flat = distance_matrix.view(B, -1)  # B x (n*n)
    distances = torch.gather(distance_flat, 1, linear_indices.view(B, K * n)).view(B, K, n)

    total_distance = distances.sum(dim=2)  # B x K
    return total_distance, decoded_tours

def decode(positions):
    decoded_tours = torch.argsort(positions, dim=1, descending=True)  # B x n

    return decoded_tours

def train_model(model, train_loader, num_cities, pso_iterations=25, epochs=50, batch_size=16, lr=1e-4, gamma=0.99, device='cuda', save_path="psognn.pth"):
    model.to(device)
    optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

    for epoch in range(epochs):
        model.train()
        total_pso_loss = 0.0
        total_policy_loss = 0.0
        batch_count = 0

        for batch in train_loader:
            batch_count += 1
            city_positions = batch['coordinates'].to(device)  # B x n x 2
            distance_matrices = batch['distance_matrix'].to(device)  # B x n x n

            # Initialize positions và velocities
            B, n = city_positions.size(0), num_cities
            K = n  # Number of solutions per problem
            positions = torch.rand((B, K, n), dtype=torch.float32, device=device)  # B x K x n
            velocities = torch.zeros_like(positions, device=device)  # B x K x n

            # Compute initial p_best
            p_best_fitness, _ = compute_fitness_loss(positions, distance_matrices)  # B x K
            p_best_positions = positions.clone().detach()  # B x K x n

            # Compute initial g_best
            g_best_fitness, g_best_indices = p_best_fitness.min(dim=1)  # B
            g_best_position = positions[torch.arange(B, device=device), g_best_indices]  # B x n

            # Initialize lists to store log probabilities và rewards
            log_probs_list = []
            rewards_list = []

            for iteration in range(pso_iterations):
                # Tạo ActionSampler mới trong mỗi iteration
                sampler = ActionSampler(model)

                # Sample actions tại mỗi iteration
                actions, log_probs = sampler.sample_action(city_positions, distance_matrices, positions)  # B x K x 3, B x K
                # print(f"Step: {iteration} action {actions}, log_probs {log_probs}")

                # Tách các tham số hành động từ actions
                w = actions[:, :, 0].unsqueeze(-1)  # B x K x 1
                c1 = actions[:, :, 1].unsqueeze(-1)  # B x K x 1
                c2 = actions[:, :, 2].unsqueeze(-1)  # B x K x 1

                # Tính toán điều kiện hội tụ
                convergence_condition = (1 + w - c1 - c2) ** 2  # Điều kiện hội tụ
                penalty_threshold = 4 * w  # Ngưỡng phạt

                # Cấp phần thưởng/phạt dựa trên điều kiện hội tụ
                convergence_rewards = torch.where(
                    convergence_condition < penalty_threshold,
                    torch.tensor(10, device=positions.device),  # Phần thưởng
                    torch.tensor(-5, device=positions.device)  # Phạt
                ).squeeze(-1)

                # Cập nhật velocities và positions
                velocities = (
                    w * velocities +
                    c1 * (p_best_positions - positions) +
                    c2 * (g_best_position.unsqueeze(1) - positions)  # B x K x n
                ) 
                # + 0.001 * torch.randn_like(velocities)
                positions = positions + velocities 
                positions = torch.sigmoid(positions)
                positions = positions.detach()
                fitness, decoded_tours = compute_fitness_loss(positions, distance_matrices)  # B x K, B x K x n
                log_probs_list.append(log_probs)  # List of B x K

                # Kết hợp reward từ hội tụ và loss
                rewards_list.append(-fitness)  # List of B x K

                # Cập nhật p_best
                better_mask = fitness < p_best_fitness  # B x K
                p_best_positions = torch.where(better_mask.unsqueeze(-1), positions, p_best_positions)  # B x K x n
                p_best_fitness = torch.min(fitness, p_best_fitness)  # B x K

                # Cập nhật g_best
                g_best_fitness_new, g_best_indices_new = fitness.min(dim=1)  # B
                update_mask = g_best_fitness_new < g_best_fitness  # B
                g_best_fitness = torch.min(g_best_fitness, g_best_fitness_new)  # B
                g_best_position_new = positions[torch.arange(B, device=device), g_best_indices_new]  # B x n

                g_best_position = torch.where(
                    update_mask.unsqueeze(-1),
                    g_best_position_new,
                    g_best_position
                )  # B x n

            # Stack log_probs và rewards
            log_probs_tensor = torch.cat(log_probs_list, dim=1)  # B x (K * pso_iterations)
            rewards_tensor = torch.cat(rewards_list, dim=1)      # B x (K * pso_iterations)
            rewards_tensor = (rewards_tensor - rewards_tensor.mean()) / (rewards_tensor.std() + 1e-8)
            # rewards_tensor = torch.clamp(rewards_tensor, min=-400.0, max=400.0)
            # Compute discounted rewards
            discounted_rewards = rewards_tensor.clone()
            for t in range(rewards_tensor.size(1) - 2, -1, -1):
                discounted_rewards[:, t] += gamma * discounted_rewards[:, t + 1]

            policy_loss = -(log_probs_tensor * discounted_rewards).mean()
            pso_loss = g_best_fitness.mean()  # Scalar
            # total_loss = policy_loss + pso_loss
            optimizer.zero_grad()
            policy_loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()

            # Accumulate losses
            total_pso_loss += pso_loss.item()
            total_policy_loss += policy_loss.item()

        # Compute average losses cho epoch
        avg_pso_loss = total_pso_loss / batch_count
        avg_policy_loss = total_policy_loss / batch_count

        scheduler.step(avg_pso_loss)
        print(f"Epoch {epoch+1}/{epochs}, Average PSO Loss: {avg_pso_loss:.4f}, Average Policy Loss: {avg_policy_loss:.4f}\n")

    # Training Runner
def run_training(num_cities=20, train_data_path="train", batch_size=16, lr=1e-4, epochs=50, pso_iterations=25, save_path="psognn.pth", device="cuda"):
    print("Loading training data...")
    train_loader = create_dataloader(train_data_path, num_cities, batch_size, shuffle=True)

    print("Initializing model...")
    model = CityGNN(input_dim=2, hidden_dim=128, output_dim=3, num_cities=num_cities)

    print("Starting training...")
    train_model(
        model=model,
        train_loader=train_loader,
        num_cities=num_cities,
        pso_iterations=pso_iterations,
        epochs=epochs,
        batch_size=batch_size,
        lr=lr,
        device=device,
        save_path=save_path
    )
    print("Training complete.")

# Example usage
if __name__ == "__main__":
    run_training(
        num_cities=20,
        train_data_path="/kaggle/input/20-citi/train",
        batch_size=32,
        lr=1e-4,
        epochs=30,
        pso_iterations=50,
        save_path="/kaggle/working/psognn.pth",
        device="cuda" if torch.cuda.is_available() else "cpu"
    )


In [ ]:
import torch
import numpy as np

def test_model(model, test_loader, num_cities, pso_iterations=5, device="cuda", seed=625):
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    model.to(device)
    model.eval()

    all_best_fitness = []  # Lưu giá trị tốt nhất của tất cả bài toán
    results = []  # Lưu kết quả của từng bài toán

    with torch.no_grad():
        for batch_id, batch in enumerate(test_loader):
            city_positions = batch['coordinates'].to(device)  # B x n x 2
            distance_matrices = batch['distance_matrix'].to(device)  # B x n x n

            B, n = city_positions.size(0), num_cities
            K = n  # Số lời giải cho mỗi bài toán

            best_fitness_per_problem = []

            for _ in range(pso_iterations):
                # Initialize positions and velocities
                positions = torch.rand((B, K, n), dtype=torch.float32, device=device)  # B x K x n
                velocities = torch.zeros_like(positions, device=device)  # B x K x n

                # Compute initial g_best
                fitness, _ = compute_fitness_loss(positions, distance_matrices)  # B x K
                g_best_fitness, g_best_indices = fitness.min(dim=1)  # B
                g_best_position = positions[torch.arange(B, device=device), g_best_indices]  # B x n

                for iteration in range(pso_iterations):
                    # Tạo ActionSampler mới trong mỗi iteration
                    sampler = ActionSampler(model)

                    # Sample actions tại mỗi iteration
                    actions, _ = sampler.sample_action(city_positions, distance_matrices, positions)  # B x K x 3

                    # Tách các tham số hành động từ actions
                    w = actions[:, :, 0].unsqueeze(-1)  # B x K x 1
                    c1 = actions[:, :, 1].unsqueeze(-1)  # B x K x 1
                    c2 = actions[:, :, 2].unsqueeze(-1)  # B x K x 1

                    # Cập nhật velocities và positions
                    velocities = (
                        w * velocities +
                        c1 * (positions - g_best_position.unsqueeze(1)) +
                        c2 * (g_best_position.unsqueeze(1) - positions)
                    )
                    positions = positions + velocities
                    positions = torch.sigmoid(positions)

                    # Tính fitness
                    fitness, _ = compute_fitness_loss(positions, distance_matrices)  # B x K

                    # Cập nhật g_best
                    g_best_fitness_new, g_best_indices_new = fitness.min(dim=1)  # B
                    update_mask = g_best_fitness_new < g_best_fitness  # B
                    g_best_fitness = torch.min(g_best_fitness, g_best_fitness_new)  # B
                    g_best_position_new = positions[torch.arange(B, device=device), g_best_indices_new]  # B x n

                    g_best_position = torch.where(
                        update_mask.unsqueeze(-1),
                        g_best_position_new,
                        g_best_position
                    )  # B x n

                # Lưu lại fitness tốt nhất cho mỗi lần chạy
                best_fitness_per_problem.append(g_best_fitness.cpu().numpy())

            # Tính toán các giá trị cần thiết cho từng bài toán
            best_fitness_per_problem = np.array(best_fitness_per_problem)  # (pso_iterations x B)
            best_fitness = best_fitness_per_problem.min(axis=0)  # Giá trị tốt nhất từng bài toán
            worst_fitness = best_fitness_per_problem.max(axis=0)  # Giá trị tệ nhất từng bài toán
            avg_fitness = best_fitness_per_problem.mean(axis=0)  # Giá trị trung bình từng bài toán

            # Lưu kết quả
            for i in range(B):
                results.append({
                    "problem_id": batch_id * B + i,
                    "best_fitness": best_fitness[i],
                    "worst_fitness": worst_fitness[i],
                    "avg_fitness": avg_fitness[i]
                })

            all_best_fitness.extend(best_fitness)

    # Tính giá trị trung bình tốt nhất của tất cả các bài toán
    overall_best_avg = np.mean(all_best_fitness)

    # In kết quả
    for result in results:
        print(f"Problem {result['problem_id']} - Best: {result['best_fitness']:.4f}, "
              f"Worst: {result['worst_fitness']:.4f}, Avg: {result['avg_fitness']:.4f}")

    print(f"\nOverall Average Best Fitness: {overall_best_avg:.4f}")
    return results, overall_best_avg
def run_test(model, test_loader, num_cities, seed=625):
    print("Running test...")
    results, overall_best_avg = test_model(
        model=model,
        test_loader=test_loader,
        num_cities=num_cities,
        pso_iterations=25,
        device="cuda" if torch.cuda.is_available() else "cpu",
        seed=seed
    )
    print("Test completed.")
    return results, overall_best_avg
if __name__ == "__main__":
    from torch.utils.data import DataLoader

    # Giả định rằng TSPDataset và mô hình đã được định nghĩa từ trước
    test_dataset = "/kaggle/input/20-test/test"
    test_loader = create_dataloader(test_dataset, num_cities = 20, batch_size = 32, shuffle=True)

    # Tạo một mô hình giả định đã được huấn luyện
    model = CityGNN(input_dim=2, hidden_dim=128, output_dim=3, num_cities=20)

    # Chạy test
    results, overall_best_avg = run_test(
        model=model,
        test_loader=test_loader,
        num_cities=20,
        seed=625
    )

In [ ]:
import torch
import numpy as np

def test_model(model, test_loader, num_cities, pso_iterations=5, device="cuda", seed=None):
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    model.to(device)
    model.eval()

    all_best_fitness = []  # Lưu giá trị tốt nhất của tất cả bài toán
    results = []  # Lưu kết quả của từng bài toán

    with torch.no_grad():
        for batch_id, batch in enumerate(test_loader):
            city_positions = batch['coordinates'].to(device)  # B x n x 2
            distance_matrices = batch['distance_matrix'].to(device)  # B x n x n

            B, n = city_positions.size(0), num_cities
            K = n  # Số lời giải cho mỗi bài toán

            best_fitness_per_problem = []

            for _ in range(pso_iterations):
                # Initialize positions and velocities
                positions = torch.rand((B, K, n), dtype=torch.float32, device=device)  # B x K x n
                velocities = torch.zeros_like(positions, device=device)  # B x K x n

                # Compute initial g_best
                fitness, _ = compute_fitness_loss(positions, distance_matrices)  # B x K
                g_best_fitness, g_best_indices = fitness.min(dim=1)  # B
                g_best_position = positions[torch.arange(B, device=device), g_best_indices]  # B x n

                for iteration in range(pso_iterations):
                    # Tạo ActionSampler mới trong mỗi iteration
                    sampler = ActionSampler(model)

                    # Sample actions tại mỗi iteration
                    actions, _ = sampler.sample_action(city_positions, distance_matrices, positions)  # B x K x 3

                    # Tách các tham số hành động từ actions
                    w = actions[:, :, 0].unsqueeze(-1)  # B x K x 1
                    c1 = actions[:, :, 1].unsqueeze(-1)  # B x K x 1
                    c2 = actions[:, :, 2].unsqueeze(-1)  # B x K x 1

                    # Cập nhật velocities và positions
                    velocities = (
                        w * velocities +
                        c1 * (positions - g_best_position.unsqueeze(1)) +
                        c2 * (g_best_position.unsqueeze(1) - positions)
                    )
                    positions = positions + velocities
                    positions = torch.sigmoid(positions)

                    # Tính fitness
                    fitness, _ = compute_fitness_loss(positions, distance_matrices)  # B x K

                    # Cập nhật g_best
                    g_best_fitness_new, g_best_indices_new = fitness.min(dim=1)  # B
                    update_mask = g_best_fitness_new < g_best_fitness  # B
                    g_best_fitness = torch.min(g_best_fitness, g_best_fitness_new)  # B
                    g_best_position_new = positions[torch.arange(B, device=device), g_best_indices_new]  # B x n

                    g_best_position = torch.where(
                        update_mask.unsqueeze(-1),
                        g_best_position_new,
                        g_best_position
                    )  # B x n

                # Lưu lại fitness tốt nhất cho mỗi lần chạy
                best_fitness_per_problem.append(g_best_fitness.cpu().numpy())

            # Tính toán các giá trị cần thiết cho từng bài toán
            best_fitness_per_problem = np.array(best_fitness_per_problem)  # (pso_iterations x B)
            best_fitness = best_fitness_per_problem.min(axis=0)  # Giá trị tốt nhất từng bài toán
            worst_fitness = best_fitness_per_problem.max(axis=0)  # Giá trị tệ nhất từng bài toán
            avg_fitness = best_fitness_per_problem.mean(axis=0)  # Giá trị trung bình từng bài toán

            # Lưu kết quả
            for i in range(B):
                results.append({
                    "problem_id": batch_id * B + i,
                    "best_fitness": best_fitness[i],
                    "worst_fitness": worst_fitness[i],
                    "avg_fitness": avg_fitness[i]
                })

            all_best_fitness.extend(best_fitness)

    # Tính giá trị trung bình tốt nhất của tất cả các bài toán
    overall_best_avg = np.mean(all_best_fitness)

    # In kết quả
    for result in results:
        print(f"Problem {result['problem_id']} - Best: {result['best_fitness']:.4f}, "
              f"Worst: {result['worst_fitness']:.4f}, Avg: {result['avg_fitness']:.4f}")

    print(f"\nOverall Average Best Fitness: {overall_best_avg:.4f}")
    return results, overall_best_avg


# Example code to run the test_model
def run_test(model, test_loader, num_cities, seed=None, load_path=None):
    if load_path:
        print(f"Loading model from {load_path}...")
        model.load_state_dict(torch.load(load_path))
        print("Model loaded successfully.")

    print("Running test...")
    results, overall_best_avg = test_model(
        model=model,
        test_loader=test_loader,
        num_cities=num_cities,
        pso_iterations=5,
        device="cuda" if torch.cuda.is_available() else "cpu",
        seed=seed
    )
    print("Test completed.")
    return results, overall_best_avg


# Run example test case
if __name__ == "__main__":
    from torch.utils.data import DataLoader

    # Giả định rằng TSPDataset và mô hình đã được định nghĩa từ trước
    test_dataset = "/kaggle/input/20-test/test"
    test_loader = create_dataloader(test_dataset, num_cities = 20, batch_size = 32, shuffle=True)

    # Tạo một mô hình giả định đã được huấn luyện
    model = CityGNN(input_dim=2, hidden_dim=128, output_dim=3, num_cities=20)

    # Chạy test
    results, overall_best_avg = run_test(
        model=model,
        test_loader=test_loader,
        num_cities=20,
        seed=625,
        load_path="psognn.pth"
    )